In [1]:
import pandas as pd
import numpy as np

pd.options.display.float_format = '{:.5f}'.format

In [2]:
user_artist_data = pd.read_csv("profiledata_06-May-2005/user_artist_data.txt", sep=" ", names=['userid', 'artistid', 'playcount'])

In [6]:
artist_name = pd.read_csv("profiledata_06-May-2005/artist_data.txt", sep="\t", names=['artistid', 'artistname'], error_bad_lines=False, warn_bad_lines=False)

/tmp/ipykernel_1424/1603388667.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  artist_name = pd.read_csv("profiledata_06-May-2005/artist_data.txt", sep="\t", names=['artistid', 'artistname'], error_bad_lines=False, warn_bad_lines=False)
/tmp/ipykernel_1424/1603388667.py:1: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  artist_name = pd.read_csv("profiledata_06-May-2005/artist_data.txt", sep="\t", names=['artistid', 'artistname'], error_bad_lines=False, warn_bad_lines=False)
/tmp/ipykernel_1424/1603388667.py:1: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  artist_name = pd.read_csv("profiledata_06-May-2005/artist_data.txt", sep="\t", names=['artistid', 'artistname'], error_bad_lines=False, warn_bad_lines=False)


In [7]:
artist_name

,artistid,artistname
0,1134999,06Crazy Life
1,6821360,Pang Nakarin
2,10113088,"Terfel, Bartoli- Mozart: Don"
3,10151459,The Flaming Sidebur
4,6826647,Bodenstandig 3000
...,...,...
2100515,10434912,Ray Charles f. Natalie Cole
2100516,10299430,Maarja ja Peeter Rebane
2100517,10299431,Weezer - Undone (The Sweater Song)
2100518,10299433,Div. Norske Artister


In [10]:
import implicit

model = implicit.als.AlternatingLeastSquares(factors=2, regularization=0.1, iterations=10)

In [11]:
user_artist_data = user_artist_data[:100000]

In [12]:
user_artist_data.describe()

,userid,artistid,playcount
count,100000.00000,100000.00000,100000.00000
mean,1000279.30585,1153704.17701,12.71207
std,162.95012,1654081.52708,64.92093
min,1000002.00000,1.00000,1.00000
25%,1000145.00000,1000049.00000,1.00000
50%,1000260.00000,1007302.00000,2.00000
75%,1000405.00000,1055479.00000,7.00000
max,1000586.00000,10787933.00000,8561.00000


In [23]:
user_artist_data

,userid,artistid,playcount
0,1000002,1,55
1,1000002,1000006,33
2,1000002,1000007,8
3,1000002,1000009,144
4,1000002,1000010,314
...,...,...,...
99995,1000586,1001362,2
99996,1000586,1001365,1
99997,1000586,1001412,1
99998,1000586,1001424,4


In [18]:
from scipy.sparse import csr_matrix

my_ratings = csr_matrix((user_artist_data['playcount'], (user_artist_data['userid'], user_artist_data['artistid'])))
my_ratings

<1000587x10787934 sparse matrix of type '<class 'numpy.int64'>'
	with 100000 stored elements in Compressed Sparse Row format>

In [30]:
from implicit.evaluation import train_test_split

# (ratings, train_percentage=0.8, random_state=None)

trd, tsd = train_test_split(my_ratings, train_percentage=0.8, random_state=None)

In [21]:
trd, tsd

(<1000587x10787934 sparse matrix of type '<class 'numpy.int64'>'
 	with 80093 stored elements in Compressed Sparse Row format>,
 <1000587x10787934 sparse matrix of type '<class 'numpy.int64'>'
 	with 19907 stored elements in Compressed Sparse Row format>)

In [22]:
model.fit(trd)

100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 10.63it/s]


<1x10787934 sparse matrix of type '<class 'numpy.int64'>'
	with 665 stored elements in Compressed Sparse Row format>

In [27]:

userid = 1000002

ids, scores = model.recommend(userid, trd[userid], N=3, filter_already_liked_items=False)

ids, scores

(array([    393,     930, 1000123], dtype=int32),
 array([1.2343774, 1.1764233, 1.1341997], dtype=float32))

In [31]:
from implicit.evaluation import mean_average_precision_at_k

mean_average_precision_at_k(model, trd, tsd, K=10, show_progress=True, num_threads=1)
# (model, train_user_items, test_user_items, K=10, show_progress=True, num_threads=1)

100%|█████████████████████████████████████████| 386/386 [00:12<00:00, 29.84it/s]


0.03870147627271978